In [2]:
!pip install ifcopenshell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.9 MB/s eta 0:00:00


In [3]:
import ifcopenshell

In [4]:
# Load an IFC file
ifc_file = ifcopenshell.open("/content/1.ifc")

# Get basic file information
schema = ifc_file.schema  # e.g., 'IFC2X3', 'IFC4', etc.
print(f"IFC Schema: {schema}")

IFC Schema: IFC2X3


# Extracting Elements by Type

In [5]:
# Get all walls in the model
walls = ifc_file.by_type("IfcWall")
print(f"Found {len(walls)} walls")

# Get other element types
slabs = ifc_file.by_type("IfcSlab")
doors = ifc_file.by_type("IfcDoor")
windows = ifc_file.by_type("IfcWindow")

Found 53 walls


# Accessing Element Properties

In [6]:
# For a specific wall
wall = walls[0]

# Basic attributes
wall_id = wall.id()
global_id = wall.GlobalId
name = wall.Name if hasattr(wall, "Name") else "Unnamed"

print(f"Wall ID: {wall_id}, GlobalId: {global_id}, Name: {name}")

# Get property sets
properties = {}
if hasattr(wall, "IsDefinedBy"):
    for definition in wall.IsDefinedBy:
        if hasattr(definition, "RelatingPropertyDefinition"):
            prop_def = definition.RelatingPropertyDefinition
            if hasattr(prop_def, "HasProperties"):
                for prop in prop_def.HasProperties:
                    if hasattr(prop, "Name") and hasattr(prop, "NominalValue"):
                        if hasattr(prop.NominalValue, "wrappedValue"):
                            properties[prop.Name] = prop.NominalValue.wrappedValue

print("Properties:", properties)

Wall ID: 533, GlobalId: 1SSXz$2dn5rQLZV7vKm8tO, Name: Basic Wall:Generic - 150mm:351941
Properties: {'Reference': 'Generic - 150mm', 'IsExternal': True, 'ExtendToStructure': False, 'LoadBearing': False}


# Extracting Quantities

In [7]:
# Get quantity information
quantities = {}
if hasattr(wall, "IsDefinedBy"):
    for definition in wall.IsDefinedBy:
        if hasattr(definition, "RelatingPropertyDefinition"):
            prop_def = definition.RelatingPropertyDefinition
            if prop_def.is_a("IfcElementQuantity"):
                for quantity in prop_def.Quantities:
                    if quantity.is_a("IfcQuantityLength"):
                        quantities[quantity.Name] = quantity.LengthValue
                    elif quantity.is_a("IfcQuantityArea"):
                        quantities[quantity.Name] = quantity.AreaValue
                    elif quantity.is_a("IfcQuantityVolume"):
                        quantities[quantity.Name] = quantity.VolumeValue

print("Quantities:", quantities)

Quantities: {}


# Extracting Relationships

In [8]:
# Get spatial containment (which space contains this wall)
spaces = []
for rel in ifc_file.by_type("IfcRelContainedInSpatialStructure"):
    if wall in rel.RelatedElements:
        spaces.append(rel.RelatingStructure.Name)

print("Contained in spaces:", spaces)

# Get connected elements (e.g., walls connected to this wall)
connected_elements = []
for rel in ifc_file.by_type("IfcRelConnectsElements"):
    if rel.RelatingElement == wall:
        connected_elements.append(rel.RelatedElement)
    elif rel.RelatedElement == wall:
        connected_elements.append(rel.RelatingElement)

print(f"Connected to {len(connected_elements)} other elements")

Contained in spaces: ['Level 1']
Connected to 3 other elements


# Extracting Geometry (Advanced)

In [9]:
# Note: Geometry extraction requires additional ifcopenshell components
# This is a simplified example

try:
    # You need ifcopenshell's geometry module for this
    import ifcopenshell.geom

    # Create a geometry settings object
    settings = ifcopenshell.geom.settings()

    # Create a geometry from the wall
    shape = ifcopenshell.geom.create_shape(settings, wall)

    # Extract vertices
    verts = shape.geometry.verts
    print(f"Geometry has {len(verts)//3} vertices")

    # Extract faces
    faces = shape.geometry.faces
    print(f"Geometry has {len(faces)//3} triangular faces")

except ImportError:
    print("Geometry module not available")

Geometry has 14 vertices
Geometry has 24 triangular faces


# Complete Example to Extract All Elements to JSON

In [11]:
import ifcopenshell
import json

def extract_ifc_to_json(ifc_file_path, output_json_path):
    """Extract IFC data to a JSON file"""

    ifc_file = ifcopenshell.open(ifc_file_path)

    # Dictionary to store all extracted data
    building_data = {
        "schema": ifc_file.schema,
        "elements": []
    }

    # Element types to extract
    element_types = [
        "IfcWall", "IfcSlab", "IfcDoor", "IfcWindow",
        "IfcColumn", "IfcBeam", "IfcSpace"
    ]

    # Extract each element type
    for element_type in element_types:
        elements = ifc_file.by_type(element_type)

        for element in elements:
            element_data = {
                "id": element.id(),
                "type": element_type,
                "global_id": element.GlobalId,
                "name": element.Name if hasattr(element, "Name") else "Unnamed",
                "properties": {},
                "quantities": {}
            }

            # Extract properties
            if hasattr(element, "IsDefinedBy"):
                for definition in element.IsDefinedBy:
                    if hasattr(definition, "RelatingPropertyDefinition"):
                        prop_def = definition.RelatingPropertyDefinition

                        # Property sets
                        if hasattr(prop_def, "HasProperties"):
                            for prop in prop_def.HasProperties:
                                if hasattr(prop, "Name") and hasattr(prop, "NominalValue"):
                                    if hasattr(prop.NominalValue, "wrappedValue"):
                                        element_data["properties"][prop.Name] = str(prop.NominalValue.wrappedValue)

                        # Quantities
                        if prop_def.is_a("IfcElementQuantity"):
                            for quantity in prop_def.Quantities:
                                if hasattr(quantity, "Name"):
                                    if quantity.is_a("IfcQuantityLength"):
                                        element_data["quantities"][quantity.Name] = quantity.LengthValue
                                    elif quantity.is_a("IfcQuantityArea"):
                                        element_data["quantities"][quantity.Name] = quantity.AreaValue
                                    elif quantity.is_a("IfcQuantityVolume"):
                                        element_data["quantities"][quantity.Name] = quantity.VolumeValue
                                    elif quantity.is_a("IfcQuantityCount"):
                                        element_data["quantities"][quantity.Name] = quantity.CountValue

            building_data["elements"].append(element_data)

    # Write to JSON file
    with open(output_json_path, 'w') as f:
        json.dump(building_data, f, indent=4)

    return building_data

# Example usage
data = extract_ifc_to_json("1.ifc", "extracted_data.json")
print(f"Extracted {len(data['elements'])} elements to JSON")

Extracted 82 elements to JSON


In [12]:
from google.colab import files

# After creating your JSON file
extract_ifc_to_json("1.ifc", "extracted_data.json")

# Download the file
files.download("extracted_data.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>